In [4]:
from IPython.display import display

import numpy as np
import pandas as pd
import os
from ebmdatalab import bq
import datetime
import plotly.express as px
import plotly.io as pio
from IPython.display import Markdown
import warnings
import requests
warnings.filterwarnings('ignore')
pio.templates.default = "plotly_white"
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from pandas.plotting import register_matplotlib_converters
import scipy.stats as stats
register_matplotlib_converters()
from statsforecast import StatsForecast

from statsforecast.models import ARIMA
from time import time
import seaborn as sns
sns.set(style="whitegrid")
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from statsforecast.arima import arima_string
import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = np.random.seed(0)


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
dark_style = {
    'figure.facecolor': '#212946',
    'axes.facecolor': '#212946',
    'savefig.facecolor':'#212946',
    'axes.grid': True,
    'axes.grid.which': 'both',
    'axes.spines.left': False,
    'axes.spines.right': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False,
    'grid.color': '#2A3459',
    'grid.linewidth': '1',
    'text.color': '0.9',
    'axes.labelcolor': '0.9',
    'xtick.color': '0.9',
    'ytick.color': '0.9',
    'font.size': 12 }
plt.rcParams.update(dark_style)

from pylab import rcParams
rcParams['figure.figsize'] = (18,7)

### Get the data from BQ

In [6]:
sql = """
SELECT DATE(month) AS month,
       bnf_name,
       bnf_code,
       SUM(quantity) AS quantity,
       SUM(items) AS items,
       SUM(actual_cost) AS actual_cost
FROM   hscic.normalised_prescribing 
WHERE  month >= '2020-04-01'
GROUP BY month,
          bnf_code,
          bnf_name 
"""

exportfile = os.path.join("..","data","rx_df.csv") #defines name for cache file
rx_df = bq.cached_read(sql, csv_path=exportfile, use_cache=True) #uses BQ if changed, otherwise csv cache file
rx_df['month'] = pd.to_datetime(rx_df['month']) #ensure dates are in datetimeformat

In [7]:
#create test dataframe with three year's data

start_date = '2020-04-01'
end_date = '2023-03-01'

# Filter the DataFrame between the two dates
test_rx_df = rx_df[(rx_df['month'] >= start_date) & (rx_df['month'] <= end_date)]


### Make data containing top % either by items or cost

In [8]:
# Define the top x% you want to identify
top_x_percent = 15 

# Group by 'bnf_code' and aggregate sum of 'items' and 'actual_cost'
grouped_df = test_rx_df.groupby('bnf_code').agg({'items': 'sum', 'actual_cost': 'sum'}).reset_index()

# Sort by 'items' to get the cumulative sum for items
grouped_df = grouped_df.sort_values('items', ascending=False)

# Calculate the cumulative sum and the total sum for items
grouped_df['cumulative_items'] = grouped_df['items'].cumsum()
total_items = grouped_df['items'].sum()

# Identify the threshold value for the top x% of items
threshold_items = total_items * (top_x_percent / 100)

# Find the bnf_codes that contribute to the top x% of items
top_items_bnf_codes = grouped_df[grouped_df['cumulative_items'] <= threshold_items]['bnf_code']

# Repeat the same process for 'actual_cost'
grouped_df = grouped_df.sort_values('actual_cost', ascending=False)
grouped_df['cumulative_actual_cost'] = grouped_df['actual_cost'].cumsum()
total_actual_cost = grouped_df['actual_cost'].sum()
threshold_actual_cost = total_actual_cost * (top_x_percent / 100)

top_actual_cost_bnf_codes = grouped_df[grouped_df['cumulative_actual_cost'] <= threshold_actual_cost]['bnf_code']

# Combine both sets of bnf_codes (union)
top_bnf_codes = set(top_items_bnf_codes).union(set(top_actual_cost_bnf_codes))

# Filter the original filtered_df for these top bnf_codes
final_df = test_rx_df[test_rx_df['bnf_code'].isin(top_bnf_codes)]
print(len(top_bnf_codes))

21


In [9]:
# let's try stats forecast


In [10]:

final_df.head()

,month,bnf_name,bnf_code,quantity,items,actual_cost
1875,2022-08-01,Apixaban 5mg tablets,0208020Z0AAABAB,27516822.0,490424,2.444558e+07
2849,2022-08-01,Linagliptin 5mg tablets,0601023AEAAAAAA,5705550.0,219089,6.343304e+06
3175,2022-08-01,FreeStyle Libre 2 Sensor,21480000101,315483.0,140478,1.032079e+07
3633,2022-08-01,Lansoprazole 30mg gastro-resistant capsules,0103050L0AAAAAA,52126428.0,1633403,1.916921e+06
4519,2022-08-01,Dapagliflozin 10mg tablets,0601023AGAAABAB,7526380.0,251721,9.199135e+06


### Using Statsforecast

In [52]:
missing_df = final_df[['month', 'bnf_code', 'quantity']].copy()

#Determine the minimum and maximum months in the DataFrame
min_month = missing_df['month'].min()
max_month = missing_df['month'].max()

# Create a complete list of months and bnf_codes
complete_months = pd.date_range(start=min_month, end=max_month, freq='MS').strftime('%Y-%m').tolist()
complete_bnf_codes = missing_df['bnf_code'].unique().tolist()

#Create a MultiIndex from the complete list of months and bnf_codes
multi_index = pd.MultiIndex.from_product([complete_months, complete_bnf_codes], names=['month', 'bnf_code'])

#Reindex the original DataFrame to fill in missing combinations
df = missing_df.set_index(['month', 'bnf_code']).reindex(multi_index, fill_value=0).reset_index()

# 4. Rename columns
df.columns = ['ds', 'unique_id', 'y']

# Now df_complete will have all combinations of month and bnf_code with 0 quantity where there was no activity
print(df)

           ds        unique_id           y
0     2020-04  0208020AAAAABAB   1217058.0
1     2020-04  0302000C0BQABBZ    106152.0
2     2020-04  0209000C0AAAAAA  24257003.0
3     2020-04  0302000C0BPABBF    314558.0
4     2020-04  0407010F0AAAHAH  51142453.0
...       ...              ...         ...
2083  2023-03  0208020Y0AAACAC   7674187.0
2084  2023-03  0208020Y0AAABAB   2276577.0
2085  2023-03  0208020Z0AAAAAA  11569065.0
2086  2023-03  0301011ABBBAAA0    198563.0
2087  2023-03  1404000H0AAANAN    107136.0

[2088 rows x 3 columns]


In [53]:
# create statsforecast model
season_length = 12 # Monthly data 
horizon = 12 # number of predictions
models = [AutoARIMA(season_length=season_length)] # set AutoARIMA as model
sf = StatsForecast(df=df,
                   models=models,
                   freq='MS', #frequency start of month
                   n_jobs=-1) # use all processors

#### Create train and test models

In [54]:
Y_train_df = df[df.ds<='2022-03-01'] 
Y_test_df = df[df.ds>='2022-04-01']

In [55]:
Y_hat_df = sf.forecast(horizon, fitted=True, level=[95])

Y_hat_df.head()

,ds,AutoARIMA,AutoARIMA-lo-95,AutoARIMA-hi-95
unique_id,,,,
0103050L0AAAAAA,2023-04-01,49827796.0,46024512.0,53631080.0
0103050L0AAAAAA,2023-05-01,55539220.0,51735936.0,59342504.0
0103050L0AAAAAA,2023-06-01,51054192.0,47250908.0,54857476.0
0103050L0AAAAAA,2023-07-01,50768192.0,46964904.0,54571476.0
0103050L0AAAAAA,2023-08-01,55003012.0,51199728.0,58806296.0


In [56]:
merged_df = pd.merge(rx_df, Y_hat_df, left_on=['month', 'bnf_code'], right_on=['ds', 'unique_id'], how='left')
merged_df = merged_df.sort_values(by=['month', 'bnf_code'])
filtered_merged_df = merged_df[merged_df['bnf_code'].isin(top_bnf_codes)]

In [57]:
filtered_merged_df.head()

,month,bnf_name,bnf_code,quantity,items,actual_cost,ds,AutoARIMA,AutoARIMA-lo-95,AutoARIMA-hi-95
385517,2020-04-01,Lansoprazole 30mg gastro-resistant capsules,0103050L0AAAAAA,45497284.0,1445097,1.972333e+06,NaT,NaN,NaN,NaN
945636,2020-04-01,Lansoprazole 15mg gastro-resistant capsules,0103050L0AAABAB,28956993.0,871221,9.591219e+05,NaT,NaN,NaN,NaN
946780,2020-04-01,Omeprazole 20mg gastro-resistant capsules,0103050P0AAAAAA,94395040.0,2483007,3.393507e+06,NaT,NaN,NaN,NaN
946854,2020-04-01,Creon 25000 gastro-resistant capsules,0109040N0BDABAQ,8747623.0,32795,2.298809e+06,NaT,NaN,NaN,NaN
393491,2020-04-01,Bisoprolol 2.5mg tablets,0204000H0AAAJAJ,25417612.0,810294,7.682165e+05,NaT,NaN,NaN,NaN


In [3]:

# Iterate through each unique bnf_code
for code in filtered_merged_df['bnf_code'].unique():
    # Filter the DataFrame for the current bnf_code
    filtered_df = filtered_merged_df[filtered_merged_df['bnf_code'] == code]
    
    # Get the corresponding bnf_name (assuming it's the same for the given bnf_code)
    bnf_name = filtered_df['bnf_name'].iloc[0]
    
    # Plot month vs items
    plt.figure(figsize=(10, 6))
    plt.plot(filtered_df['month'], filtered_df['quantity'])
    plt.plot(filtered_df['month'], filtered_df['AutoARIMA'])
    
    # Set the title to the bnf_name
    plt.title(bnf_name)
    plt.ylim(bottom=0)
    
    # Label the axes
    plt.xlabel('Month')
    plt.ylabel('Items')
    
    # Show the plot
    plt.show()


NameError: name 'filtered_merged_df' is not defined

In [ ]:
sql = """
SELECT DATE(month) AS month,
       bnf_name,
       bnf_code,
       SUM(quantity) AS quantity,
       SUM(items) AS items,
       SUM(actual_cost) AS actual_cost
FROM   hscic.normalised_prescribing 
WHERE  month >= '2020-04-01'
GROUP BY month,
          bnf_code,
          bnf_name 
"""

exportfile = os.path.join("..","data","rx_df.csv") #defines name for cache file
rx_df = bq.cached_read(sql, csv_path=exportfile, use_cache=True) #uses BQ if changed, otherwise csv cache file
rx_df['month'] = pd.to_datetime(rx_df['month']) #ensure dates are in datetimeformat